In [1]:
from numpy import asarray
from numpy import exp
from numpy.random import randn
from numpy.random import rand
from numpy.random import seed
from matplotlib import pyplot
import pandas as pd

import numpy as np
import pandas as pd
import random as rd
from random import randint
import matplotlib.pyplot as plt
from math import  floor
from itertools import cycle

In [2]:
import random as rd

In [3]:
import numpy as np
# Define constants
num_rooms = 15
num_days = 10
num_surgeries = 671
capacity = 480
threshold = 50

# Read dataset
df_surgeries = pd.read_csv('Dataset4.txt',header=None, skiprows=[0], delim_whitespace=True, usecols=[1])
time_surgeries = df_surgeries[1].values.tolist()
time_surgeries = np.array(time_surgeries)

In [4]:
def greedy(pop_size,num_surgeries,num_days,num_rooms,time_surgery):
    population = []
    REPLACE_WITH = 1
    for i in range(pop_size):
        sample = []              
        current_capacity = np.zeros(num_days*num_rooms).astype(int)  
        room_day_array = list(range(num_days*num_rooms))
        for i in range(num_surgeries):
            surgery = np.zeros(num_days*num_rooms).astype(int)
            room_day = rd.choice(room_day_array)
            if current_capacity[room_day] < capacity:
                surgery.flat[room_day] = REPLACE_WITH
                sample.append(surgery)
                current_capacity[room_day] += time_surgery[i]
            else:
                room_day_array.remove(room_day)
                room_day = rd.choice(room_day_array)
                surgery.flat[room_day] = REPLACE_WITH
                sample.append(surgery)
                current_capacity[room_day] += time_surgery[i] 
            
        population.append(sample)
    population = np.asarray(population)
    return population

In [5]:
#new greedy amazineee~! 


def greedy(pop_size,num_surgeries,num_days,num_rooms,time_surgery,threshold):
    population = []
    REPLACE_WITH = 1
    for i in range(pop_size):
        sample = []              
        current_capacity = np.zeros(num_days*num_rooms).astype(int)  
        room_day_array = list(range(num_days*num_rooms))
        for i in range(num_surgeries):
            counter=0
            surgery = np.zeros(num_days*num_rooms).astype(int)
            while (sum(surgery)==0):
                room_day = rd.choice(room_day_array)
                if (current_capacity[room_day]+time_surgery[i]) < (capacity+threshold):
                    surgery.flat[room_day] = REPLACE_WITH
                    sample.append(surgery)
                    current_capacity[room_day] += time_surgery[i]
                    break
                else:
                    counter +=1
                    if counter>=(num_days*num_rooms):                        
                        capacity_order = np.argsort(np.argsort(current_capacity))
                        index_min_capacity = min(range(len(capacity_order)), key=capacity_order.__getitem__)
                        surgery.flat[index_min_capacity] = REPLACE_WITH
                        sample.append(surgery)
                        current_capacity[index_min_capacity] += time_surgery[i]             
        population.append(sample)
    population = np.asarray(population)
    return population

In [6]:
pop_size = 1
initial_population = greedy(pop_size,num_surgeries,num_days,num_rooms,time_surgeries, threshold)
initial_population

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]])

In [7]:
#objective function
def cal_fitness(time_surgeries, population):
    fitness = np.zeros(population.shape[0])
    for i in range(population.shape[0]):
        time = np.zeros(shape=population.shape[2])
        for j in range(population.shape[1]):
            for k in range(population.shape[2]):
                time[k] += population[i][j][k] * time_surgeries[j]
        time = time-capacity
        time_day = np.split(time,num_days)
        for day in range(num_days):
            maxim_day =  max(time_day[day])
            fitness[i] += max(maxim_day,0)
    return fitness.astype(int)  

In [8]:
#invert solution
filename = 'SolutionDataset4Group3_4.txt'

initial_population = np.zeros((1,num_surgeries,num_days*num_rooms)).astype(int)

with open(filename) as file:
    lines = file.readlines()[2:]
    for line,i in zip(lines,range(len(lines))):
        int_list = [int(i) for i in line.split()]    
        j = int(int_list[2])-1+num_rooms*(int(int_list[1])-1)
        initial_population[0][i][j] = 1


In [9]:
initial_score = cal_fitness(time_surgeries, initial_population)
initial_score

array([96])

In [10]:
#swapping
import math as m
import time
t0= time.process_time()

r = 0.01
C = max(time_surgeries)
T0 = C*r
M = num_surgeries
alpha = 0.95
T = T0
COMB = 434778*10e20 #m.comb(num_surgeries, 2)
S = alpha**(m.log(COMB))
print('T',T)
print('S',S)


best_score = initial_score
current_score = initial_score
best_solution = initial_population
current_solution = initial_population

for replication in range(25):    
    print(replication)
    T = T0
    while(T > S):
        for iterations in range(M):
            idx = range(num_surgeries)
            i1, i2 = rd.sample(idx,2)
            #print(i1, i2, 'swapped rows')
            neighbor = np.copy(current_solution)
            neighbor[0][i1] = current_solution[0][i2]
            neighbor[0][i2] = current_solution[0][i1]
            #print(current_solution, 'current_solution')
            #print(neighbor, 'neighbor')
            #print(current_solution-neighbor, 'current_solution-neighbor')
            neighbor_score = cal_fitness(time_surgeries, neighbor)
            if (neighbor_score < best_score):
                best_score = neighbor_score
                best_solution = neighbor
                current_score = best_score
                current_solution = best_solution
            elif neighbor_score < current_score:
                current_score = neighbor_score
                current_solution = neighbor
            elif rd.random() < m.exp(-abs(current_score-neighbor_score)/T):
                current_score = neighbor_score
                current_solution = neighbor
                #print("Yay you jumped",e
                print("new current", current_score)
                print("best score", best_score)
                print(T, 'Temperature')
        T = alpha*T 
t1 = time.process_time()
print(t1-t0)

T 3.49
S 0.04301654244763175
0
new current [96]
best score [96]
3.49 Temperature
new current [104]
best score [96]
3.49 Temperature
new current [104]
best score [96]
3.49 Temperature
new current [104]
best score [96]
3.49 Temperature
new current [104]
best score [96]
3.49 Temperature
new current [104]
best score [96]
3.49 Temperature
new current [105]
best score [96]
3.49 Temperature
new current [105]
best score [96]
3.49 Temperature
new current [105]
best score [96]
3.49 Temperature
new current [105]
best score [96]
3.49 Temperature
new current [105]
best score [96]
3.49 Temperature
new current [105]
best score [96]
3.49 Temperature
new current [106]
best score [96]
3.49 Temperature
new current [106]
best score [96]
3.49 Temperature
new current [106]
best score [96]
3.49 Temperature
new current [113]
best score [96]
3.49 Temperature
new current [113]
best score [96]
3.49 Temperature
new current [113]
best score [96]
3.49 Temperature
new current [116]
best score [96]
3.49 Temperature
n

KeyboardInterrupt: 

In [ ]:
best_solution

In [ ]:
best_score

# best_solution

In [ ]:
%%capture cap --no-stderr

objective_value = (best_score)

print('Obj',objective_value[0])
print('Surgery','Day','OR')

for i in range(num_surgeries):
    for j in range(num_rooms*num_days):
        if best_solution[0][i][j] == 1:
            day = floor(j/num_rooms)+1
            room = (j-num_rooms*(day-1))+1
            print(i+1,day,room)

In [ ]:
#output as .txt
with open('SolutionDataset4Group3_5.txt', 'w') as f:
    f.write(cap.stdout)

In [ ]:
print('final_matrix', best_solution)

with open('final_matrix_dataset4_5.txt', 'w' ) as f:
    f.write(cap.stdout)

In [ ]:
def worst_rooms_day(time_surgeries, amount_top, population):
    fitness = np.zeros(population.shape[0])
    time = np.zeros(shape=population.shape[2])
    for j in range(population.shape[1]):
        for k in range(population.shape[2]):
            time[k] += population[i][j][k] * time_surgeries[j]
        time = time-capacity
        top_order = np.argpartition(time, amount_top)
        days_rooms_with_worst_fitness = top_order[-amount_top:]
    return days_rooms_with_worst_fitness

In [ ]:
def top_days(time_surgeries, population):
    overtime = np.zeros(shape=num_days)
    time = np.zeros(shape=population.shape[1])
    num_days_2 = int(num_days/2)
    for j in range(population.shape[0]):
        for k in range(population.shape[1]):
            time[k] += population[j][k] * time_surgeries[j]
    time = time-capacity
    for day in range(num_days):
        overtime[day] = max(max(np.split(time,num_days)[day]),0)
    top_order = np.argpartition(overtime, int(num_days_2))
    days_with_best_fitness = top_order[:int(num_days_2)]
    return days_with_best_fitness